# Construct Master Dataset by linking different tables in MIMIC-IV-ED

Note:  MIMIC-ED should be downloaded at “data/ed" folder


## Prepare Python library and raw data

In [2]:
import argparse
from helpers import *
from medcode_utils import commorbidity, extract_icd_list

In [6]:
from dataset_path import mimic_iv_path, output_path

In [34]:
complaint_dict = {"chiefcom_chest_pain" : "chest pain", "chiefcom_abdominal_pain" : "abdominal pain|abd pain", 
"chiefcom_headache" : "headache|lightheaded", "chiefcom_shortness_of_breath" : "breath", "chiefcom_back_pain" : "back pain", "chiefcom_cough" : "cough", 
"chiefcom_nausea_vomiting" : "nausea|vomit", "chiefcom_fever_chills" : "fever|chill", "chiefcom_syncope" :"syncope", "chiefcom_dizziness" : "dizz"}

## Defining health utilization timerange parameters in days
icu_transfer_timerange = 12 # hours
# past_ed_visits_timerange = 365
# past_admissions_timerange = 365 
# past_icu_visits_timerange = 365
next_ed_visit_timerange = 3
pd.set_option('display.max_columns', 100)


## Load raw data tables through pandas library

In [11]:
## Reading main tables
df_edstays = read_edstays_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/ed/edstays.csv.gz")
df_patients = read_patients_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/hosp/patients.csv.gz")
df_admissions = read_admissions_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/hosp/admissions.csv.gz")
df_icustays = read_icustays_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/icu/icustays.csv.gz")
df_triage = read_triage_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/ed/triage.csv.gz")
df_vitalsign = read_vitalsign_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/ed/vitalsign.csv.gz")
df_pyxis = read_pyxis_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/ed/pyxis.csv.gz")
df_medrecon = read_pyxis_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/ed/medrecon.csv.gz")
df_diagnoses = read_diagnoses_table("C:/Users/User/OneDrive - University of Cyprus/Desktop/Emergency_Department_Triage/Data/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz")


## ED root table, demographic and outcomes

In [13]:
## Merging patients -> merging admissions -> merging triage -> master
df_master = merge_edstays_patients_on_subject(df_edstays ,df_patients)
df_master = merge_edstays_admissions_on_subject(df_master ,df_admissions)

In [14]:
## Adding age, mortality and ICU transfer outcome
df_master = add_age(df_master)
df_master = add_inhospital_mortality(df_master)
df_master = add_ed_los(df_master)read_edstays_table
df_master = add_outcome_icu_transfer(df_master, df_icustays, icu_transfer_timerange)
df_master['outcome_hospitalization'] = ~pd.isnull(df_master['hadm_id'])
df_master['outcome_critical'] = df_master['outcome_inhospital_mortality'] | df_master[''.join(['outcome_icu_transfer_', str(icu_transfer_timerange), 'h'])]

# Sort Master table for further process
df_master = df_master.sort_values(['subject_id', 'intime']).reset_index()

# Filling subjects NA ethnicity, takes ~17s
df_master = fill_na_ethnicity(df_master)

## Health Utilization

In [16]:
## Generate past ED visits
df_master = generate_past_ed_visits(df_master, timerange=30)
df_master = generate_past_ed_visits(df_master, timerange=90)
df_master = generate_past_ed_visits(df_master, timerange=365)

In [17]:
## Oucome:  future ED revisit variables
df_master = generate_future_ed_visits(df_master, next_ed_visit_timerange)

In [18]:
## Generate past admissions
df_master = generate_past_admissions(df_master, df_admissions, timerange=30)
df_master = generate_past_admissions(df_master, df_admissions, timerange=90)
df_master = generate_past_admissions(df_master, df_admissions, timerange=365)

In [19]:
## Generate past icu visits
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=30)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=90)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=365)

In [20]:
## Generate numeric timedelta variables
df_master = generate_numeric_timedelta(df_master)

## Triage Information

In [22]:
## Mergining with triage table, Comment: revise the variable names? triage_*
df_master = merge_edstays_triage_on_subject(df_master, df_triage) ## note change to merge master 

In [23]:
## Encoding 10 chief complaints
df_master = encode_chief_complaints(df_master, complaint_dict)

## Comorbidities from diagnosis ICD

In [25]:
# This function takes about 10 min
df_master = commorbidity(df_master, df_diagnoses, df_admissions, timerange = 356*5)
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9_3digit')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v10')


Number of unique ICD codes v9:  8656
Number of unique ICD codes v9_3digit:  8597
Number of unique ICD codes v10:  14954


## ED Vital signs

In [56]:
df_master = merge_vitalsign_info_on_edstay(df_master, df_vitalsign, options=['last'])

## Medication

In [58]:
df_master = merge_med_count_on_edstay(df_master, df_pyxis)

In [59]:
df_master = merge_medrecon_count_on_edstay(df_master, df_medrecon)

## Review the master dataset and output

In [63]:
#df_master.head(100).to_csv(os.path.join(output_path, 'master_dataset_part.csv'), index=False)
# Full dataset:
df_master.to_csv(os.path.join(output_path, 'master_dataset_new.csv'), index=False)

In [69]:
 df_master

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,...,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last_x,ed_heartrate_last_x,ed_resprate_last_x,ed_o2sat_last_x,ed_sbp_last_x,ed_dbp_last_x,ed_pain_last_x,ed_temperature_last_y,ed_heartrate_last_y,ed_resprate_last_y,ed_o2sat_last_y,ed_sbp_last_y,ed_dbp_last_y,ed_pain_last_y,n_med,n_medrecon
0,0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,Other,2180,52,False,0 days 04:13:00,NaT,NaT,False,True,False,0,0,0,2180-06-26 15:54:00,50 days 16:24:00,False,0,0,0,0,0,0,4.22,NaT,50.68,98.4,70.0,16.0,97.0,106.0,63.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,Medicaid,2180,52,False,0 days 05:37:00,NaT,NaT,False,True,False,0,1,1,2180-07-22 16:24:00,25 days 18:53:00,False,0,1,1,0,0,0,5.62,NaT,25.79,98.9,88.0,18.0,97.0,116.0,88.0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,97.9,86.0,17.0,93.0,96.0,57.0,5.0,97.9,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 13:30:00,2180-07-23 14:00:00,0 days 08:06:00,True,True,True,1,2,2,2180-07-23 05:54:00,0 days 00:00:00,True,1,2,2,0,0,0,13.50,8.1,0.0,97.8,87.0,14.0,97.0,71.0,43.0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,98.2,85.0,18.0,98.0,81.0,38.0,0.0,98.2,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 08:06:00,2180-07-23 14:00:00,0 days 00:00:00,True,True,True,2,3,3,2180-08-05 20:58:00,13 days 06:58:00,False,1,2,2,0,0,0,8.10,0.0,13.29,98.7,77.0,16.0,98.0,96.0,50.0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,99.0,96.0,18.0,97.0,86.0,45.0,NaN,99.0,96.0,18.0,97.0,86.0,45.0,NaN,8.0,14.0
4,2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,Medicaid,2180,52,False,0 days 04:46:00,NaT,NaT,False,True,False,2,3,4,NaT,NaT,False,1,2,3,1,1,1,4.77,NaT,NaT,99.4,105.0,18.0,96.0,106.0,57.0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,98.1,91.0,18.0,99.0,98.0,60.0,NaN,98.1,91.0,18.0,99.0,98.0,60.0,NaN,6.0,7.0
...,...,...,...,...,...,...,...,...,..